In [1]:
# Loading the IMDB dataset
from keras.datasets import imdb
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/dados/anaconda3/lib/python3.8/site-packages/keras/datasets/imdb.py:155: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
/dados/anaconda3/lib/python3.8/site-packages/keras/datasets/imdb.py:156: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'd

In [2]:
# Encoding the integer sequences into a binary matrix
import numpy as np

def vectorize_sequences(sequences, dimension=10000):
  results = np.zeros((len(sequences), dimension))
  for i, sequence in enumerate(sequences):
    results[i, sequence] = 1.
  return results

x_train = vectorize_sequences(train_data)
x_test = vectorize_sequences(test_data)

In [3]:
# also vectorize your labels
y_train = np.asarray(train_labels).astype('float32')
y_test = np.asarray(test_labels).astype('float32')

In [4]:
import os
os.environ['PYTHONHASHSEED']=str(1)

import tensorflow as tf
import random
import numpy as np

def reset_random_seeds(SEED = 42):
  os.environ['PYTHONHASHSEED']=str(SEED)
  tf.random.set_seed(SEED)
  np.random.seed(SEED)
  random.seed(SEED)

In [7]:
from keras import models
from keras import layers

def create_and_train_model(hidden_layers=2,
                           hidden_units=16, 
                           loss_function='binary_crossentropy',
                           activation_function='relu'):
  
  model = models.Sequential()

  model.add(layers.Dense(hidden_units, activation=activation_function, input_shape=(10000,)))

  for i in range(hidden_layers - 1):
    model.add(layers.Dense(hidden_units, activation=activation_function))
 
  model.add(layers.Dense(1, activation='sigmoid'))

  model.compile(optimizer='rmsprop',
                loss=loss_function,
                metrics=['accuracy'])

  model.fit(x_train, y_train, epochs=4, batch_size=512)

  results = model.evaluate(x_test, y_test)

  return "[hidden_layers=%d, hidden_units=%d, loss_function=%s, " \
          "activation_function=%s]\n => loss: %.4f - accuracy: %.4f\n" % (
          hidden_layers, hidden_units, loss_function, activation_function,
          results[0], results[1])

In [9]:
%%time

# You used two hidden layers. Try using one or three hidden layers, 
# and see how doing so affects validation and test accuracy.
# => best result: 2 hidden layers

for value in [1, 2, 3, 4]:
  res = create_and_train_model(hidden_layers=value)
  print(res)

Epoch 1/4
49/49 [==============================] - 1s 5ms/step - loss: 0.5440 - accuracy: 0.7559
Epoch 2/4
49/49 [==============================] - 0s 5ms/step - loss: 0.2992 - accuracy: 0.9068
Epoch 3/4
49/49 [==============================] - 0s 5ms/step - loss: 0.2231 - accuracy: 0.9278
Epoch 4/4
782/782 [==============================] - 1s 582us/step - loss: 0.2799 - accuracy: 0.8893
[hidden_layers=1, hidden_units=16, loss_function=binary_crossentropy, activation_function=relu]
 => loss: 0.2799 - accuracy: 0.8893

Epoch 1/4
49/49 [==============================] - 1s 5ms/step - loss: 0.5503 - accuracy: 0.7416
Epoch 2/4
49/49 [==============================] - 0s 5ms/step - loss: 0.2740 - accuracy: 0.9051
Epoch 3/4
49/49 [==============================] - 0s 5ms/step - loss: 0.1983 - accuracy: 0.9311
Epoch 4/4
782/782 [==============================] - 1s 583us/step - loss: 0.2972 - accuracy: 0.8825
[hidden_layers=2, hidden_units=16, loss_function=binary_crossentropy, activation_fu

In [ ]:
%%time

# Try using layers with more hidden units or fewer hidden units: 
# 32 units, 64 units, and so on.

# => best result: ? hidden units

for value in [4, 8, 16, 32]:
  res = create_and_train_model(hidden_units=value)
  print(res)

In [ ]:
%%time

# Try using the mse loss function instead of binary_crossentropy.

# => best result: ? loss function

for value in ['binary_crossentropy', 'mse']:
  res = create_and_train_model(loss_function=value)
  print(res)

Epoch 1/4
49/49 [==============================] - 1s 6ms/step - loss: 0.5648 - accuracy: 0.7359
Epoch 2/4
49/49 [==============================] - 0s 7ms/step - loss: 0.2894 - accuracy: 0.9024
Epoch 3/4
49/49 [==============================] - 0s 6ms/step - loss: 0.2124 - accuracy: 0.9248
Epoch 4/4
782/782 [==============================] - 1s 1ms/step - loss: 0.2930 - accuracy: 0.8843
[hidden_layers=2, hidden_units=16, loss_function=binary_crossentropy, activation_function=relu] => loss: 0.2930 - accuracy: 0.8843


In [ ]:
%%time

# Try using the tanh activation (an activation that was popular in 
# the early days of neural networks) instead of relu.

# => best result: ? activation function

for value in ['relu', 'tanh']:
  res = create_and_train_model(activation_function=value)
  print(res)